# Cracking CMIYC_2022 Hashes (MULTIPLE HASHFILE EXAMPLE)

### Disclaimer

**As this Notebook demonstrates how to create custom hashlists/wordlists, if you run this entire notebook it will create those files in the ./challenge_files/CMIYC2022_Street directory**

### References (And Spoilers)

For more information about extracting and cracking the hashes for CMIYC 2022 contest, as well as cracking them, here are some blog posts I've written on the contest: 

- [My original writeup for compeating in the CMIYC2022 Challenges](https://reusablesec.blogspot.com/2022/08/more-password-cracking-tips-defcon-2022.html)
- 
[Revisiting the CMIYC 2022 Challenges: Using this JupyterLab framework](https://reusablesec.blogspot.com/2023/11/jupyter-lab-framework-example.html)

### Top Level Goals

I figure the best way to improve this JupyterLab framework is to try and use it myself. Beyond that though, this particular example will focus on some of the aspects that made the CMIYC2022 challenges different from the CMIYC2023 challenges. Specifically how the challenge files were plain hashes of different types, and how each dataset was generated a specific way. Therefore this guide will highlight some of the more recent features I've added into the JupyterLab framework such as the ability to automatically parse logs generated from password cracking sessions and display their results in the framework itself.

### Loading the configuration and the initial challenge files

What was nice about the CMIYC2022 challenge was that it contained several different files which only contained hashes of the same type. This makes it very easy to parse with this JupyterLab framework. In the config file, you just need to specify the "plain_hash" format. For example:

```
challenge_files:
    list14:
      file: "./challenge_files/CMIYC2022_Street/sample_hashes/list14-4214-BrunnersMentalPrisoner.hashes"
      format: "plain_hash"
      type: "mysqlna"
      source: "list14-BrunnersMentalPrisoner"
```

The 'type' is the hash format to use, and the 'source' is the name used to identify the source of the hashes in this framework.

The example below demonstrates initializing the SessionMgr to load the sample challenge hashes from the CMIYC2022 contest.

In [1]:
from lib_framework.session_mgr import SessionMgr

# Loading the config this way to make it os independent on what you are running this on
import os
import sys
config_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2022_Street',
    'config.yml'
    )

sm = SessionMgr(config_file)

Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list14-4214-BrunnersMentalPrisoner.hashes. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list16-FL_kdIZUGpI.txt. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list17.txt. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list18.hash. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list19-paidanextra500000.hashes. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list20-Authoritiesappeartohaveuncoveredavastnefariousconspiracy.hash

### Creating the "Left" password lists

While you can certainly run attacks against the raw lists of password hashes from the contest files, it can be helpful to create custom left lists to target. For example:

- Using this framework to create left lists will append the hash_id as the username for hash lists. This will make it easier to parse JtR log files.
- If you are targeting salted hash lists, creating a left list will allow you to perform further targeting of hashes to speed up cracking sessions
- Many hashes "look" the same to password crackers. E.g. MD5, MD5, NTLM, etc. Creating different files for them can speed up loading of cracking sessions as you are not loading/checking the wrong hashes.
- It's kind of nice to see these left lists get smaller over time ;p

One thing to keep in mind is you can keep re-running this after you crack more passwords and load the pots back into this framework. Reloading the pots back in is important as that way cracked passwords will not be written to the updated left list. Loading those pots will be shown later using the sm.load_main_pots() function call.

In [2]:
# Writing the left list to the CMIYC2023_Street directory
file_name = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2022_Street',
    'sample_hashes',
    'all_hashes_left.hash'
    )

sm.create_left_list(is_jtr=True, file_name=file_name);

### Performing basic analysis of cracked passwords

The next couple of commands load the pot files and perform basic analysis of progress against the challenge hashes. As you crack more passwords, you can re-run the following cells to update you on your progress.

In [3]:
sm.load_main_pots(verbose=False, update_only=True)
sm.update_main_pots()

Number of new plains added to the JtR pot file: 0
Number of new plains added to the Hashcat pot file: 0


In [4]:
sm.print_status()

Algorithm      :Total     :Cracked   :Remaining :Percentage
mysqlna        :4214      :166       :4048      :4%
half-md5       :2766      :0         :2766      :0%
raw-md5        :2933      :546       :2387      :19%
raw-sha1       :5455      :2988      :2467      :55%
raw-sha256     :4997      :0         :4997      :0%
raw-sha384     :10004     :0         :10004     :0%
mssql05        :10812     :18        :10794     :0%
ssha           :2000      :1993      :7         :100%


In [5]:
sm.print_score()

Hash Type:     Value Per Crack:    Points Earned:      Total Possible Points:
mysqlna        17                  2822                71638
half-md5       3                   0                   8298
raw-md5        1                   546                 2933
raw-sha1       5                   14940               27275
raw-sha256     13                  0                   64961
raw-sha384     46                  0                   460184
mssql05        9                   162                 97308
ssha           5                   9965                10000

Total Score: 28435
Maximum Possible Score: 742597


In [6]:
sm.print_metadata_items("source")

source                   :Number of Hashes :Cracked
list14-BrunnersMentalPrisoner:4214             :166
list16-FL_kdIZUGpI       :2766             :0
list17                   :2933             :546
list18                   :5455             :2988
list19-paidanextra500000 :4997             :0
list20-Authoritiesappearto:10004            :0
list21                   :10812            :18
list24                   :2000             :1993


### Parsing logs from password cracking sessions

Password cracking tools such as John the Ripper and Hashcat have the ability to create logs of password cracking sessions. This can be very helpful when keeping track of what attacks you have run, and how successful they were. One challenge though is the log file format for both JtR and Hashcat are very different, and they log differet peices of information. At a high level, here are some of the important info that is included, (and not included), in both log formats.

#### John the Ripper Log Format:
**Contains**
+ Detailed information about session info (e.g. command line used, attack type, wordlist dictionary, etc)
+ Information about the target hashes in a cracking session (filename, hash type, number of hashes, etc)
+ Session time and compleation info. Aka how long the attack took
+ Which rule cracked which password
**Missing**
- Cracked password hashes are not listed in the logfile. Instead it lists usernames of cracked passwords. If you don't have usernames in your target list it defaults to only listing the cracked password hash as '?'

In [7]:
sm.read_all_logs();

Unsuported Log Line: ! Invalid rule at line 743: Invalid position code ^A' accepted


**Quick Tip**: I'm splitting up the ServiceManager.read_all_logs() and the ServiceManager.print_log_sessions() calls so that if errors or issues occur it is easier to see and debug, but often you might want to put both of these in the same cell so that way you don't have to remember to run two cells in the correct order to see their results.

**Feedback Request**: If you see lines being printed out from ServiceManager.read_all_logs() that means you encountered log lines I currently don't handle. If you could submit an issue to the github repository for this toolset listing the line (or a sanatized version of it) I'd appreciate it so I can make sure I'm properly parsing it.

In [8]:
sm.print_log_sessions()

Logs for Hash Type: PDF
    Session ID: 0
        Tool: John the Ripper
        Num Cracked Hashes: 1
        Loaded: 1
        Compleated: True
        Run Time (seconds): 23
        Cracking Mode: single
            Wordlist: password.lst
            Ruleset: Wordlist
            Num Rules: 3068
            Wordlist Only Attack
Logs for Hash Type: Raw-SHA1
    Session ID: 1
        Tool: John the Ripper
        Num Cracked Hashes: 144
        Loaded: 5455
        Compleated: True
        Run Time (seconds): 18
        Cracking Mode: wordlist
            Wordlist: dic-0294.txt
            Ruleset: best64
            Num Rules: 79
    Session ID: 3
        Tool: John the Ripper
        Num Cracked Hashes: 2
        Loaded: 5455
        Compleated: True
        Run Time (seconds): 47
        Cracking Mode: wordlist
            Wordlist: hashes.org-2020.txt
    Session ID: 4
        Tool: John the Ripper
        Num Cracked Hashes: 0
        Loaded: 5455
        Compleated: False
       

### Generating rulesets from cracked passwords

A common cracking technique is to run exhausitve rulesets using a small dictioanry to identify effective rules, and then run those rules against much larger dictionaries. To do this you need to figure out which rules were effective to crack passwords. 

This framework includes the function **ServiceManager.create_ruleset(filename=None, hash_type=None, filter=None)** to do this for you.

Like the other functions in this framework, you can use the hash_type to only generate rules from successful strikes agains the target hash_type. Likewise filter will filter out strikes based on hash metadata criteria.

The function returns a Python Counter object with the rules and the number of occurences the rules were found. You can also print out the raw rules to a file using the file_name variable.

#### Dev Note:
Right now there is no cleanup with the rules, so if you mix and match John the Ripper and Hashcat cracking sessions it may output rules that don't work with the other cracking programs. There's a lot of future work here to make this process more effective.

In [9]:
# Saving the rule file this way to make it os independent on what you are running this on
file_name = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2022_Street',
    'mysqlna_cracked.rule'
    )

sm.create_ruleset(file_name=file_name, hash_type="mysqlna");